In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!pip install geopandas
!pip install geopy
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!pip install folium
import folium # map rendering library

import html5lib
!pip install lxml

     |████████████████████████████████| 931kB 7.8MB/s eta 0:00:01
     |████████████████████████████████| 10.9MB 32.6MB/s eta 0:00:01
     |████████████████████████████████| 14.7MB 51.1MB/s eta 0:00:01
     |████████████████████████████████| 1.8MB 53.8MB/s eta 0:00:01
     |████████████████████████████████| 102kB 9.0MB/s ta 0:00:011


In [2]:
# The code was removed by Watson Studio for sharing.

Your credentails:
CLIENT_ID: JMBVGR3NI4NJVGIZQ1PGHYJJBSV1ISOJRNI10IIOSVH5IU2Z
CLIENT_SECRET:V5UDKAS025L2NFAPKXDVBNCYUZVHWJ0SKEOCVM42DM1AHCE4
0.24.1


# Toronto

In [3]:
#Toronto data
wiki_url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
dflist = pd.read_html(wiki_url)[0]
#Pre-processing
#drop rows with Borough='Not assigned'
df=dflist.drop(dflist.index[dflist.Borough=='Not assigned'])
df.sort_values(by=['Postal Code'], inplace=True)
df.reset_index(drop=True, inplace=True)
!wget -q -O 'toronto_data.csv' http://cocl.us/Geospatial_data
geodf=pd.read_csv('toronto_data.csv')
df.set_index('Postal Code', inplace=True)
geodf.set_index('Postal Code',  inplace=True)
toronto_df=df.merge(geodf, left_index=True, right_index=True)
toronto_df.reset_index(inplace=True)

torontocity_data = toronto_df[toronto_df['Borough'].str.contains('Toronto')].reset_index(drop=True)
df_toronto1 = torontocity_data.drop(['Postal Code'], axis=1)
print(df_toronto1.shape)
df_toronto1.head()

(39, 4)


,Borough,Neighborhood,Latitude,Longitude
0,East Toronto,The Beaches,43.676357,-79.293031
1,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
2,East Toronto,"India Bazaar, The Beaches West",43.668999,-79.315572
3,East Toronto,Studio District,43.659526,-79.340923
4,Central Toronto,Lawrence Park,43.728020,-79.388790


In [4]:
df_toronto=df_toronto1.set_index(['Borough','Latitude','Longitude']).stack().str.split(',', expand=True).stack()
df_toronto=df_toronto.unstack(-2).reset_index(-1,drop=True).reset_index()
df_toronto = df_toronto.reindex(columns=['Borough','Neighborhood','Latitude', 'Longitude'])
print(df_toronto.shape)
df_toronto.head(78)

(78, 4)


,Borough,Neighborhood,Latitude,Longitude
0,Central Toronto,The Annex,43.672710,-79.405678
1,Central Toronto,North Midtown,43.672710,-79.405678
2,Central Toronto,Yorkville,43.672710,-79.405678
3,Central Toronto,Summerhill West,43.686412,-79.400049
4,Central Toronto,Rathnelly,43.686412,-79.400049
5,Central Toronto,South Hill,43.686412,-79.400049
6,Central Toronto,Forest Hill SE,43.686412,-79.400049
7,Central Toronto,Deer Park,43.686412,-79.400049
8,Central Toronto,Moore Park,43.689574,-79.383160
9,Central Toronto,Summerhill East,43.689574,-79.383160


In [23]:
address = 'Toronto, ON'

geolocator = Nominatim(user_agent="to_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
# create map of New York using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=12)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_toronto['Latitude'], df_toronto['Longitude'], df_toronto['Borough'], df_toronto['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [27]:
#Function to have all the venue from the different neighborhoods
LIMIT=5000
radius=500
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [7]:
toronto_venues = getNearbyVenues(names=df_toronto['Neighborhood'],
                                   latitudes=df_toronto['Latitude'],
                                   longitudes=df_toronto['Longitude']
                                  )

The Annex
 North Midtown
 Yorkville
Summerhill West
 Rathnelly
 South Hill
 Forest Hill SE
 Deer Park
Moore Park
 Summerhill East
Forest Hill North & West
 Forest Hill Road Park
Davisville
Roselawn
Davisville North
North Toronto West
 Lawrence Park
Lawrence Park
CN Tower
 King and Spadina
 Railway Lands
 Harbourfront West
 Bathurst Quay
 South Niagara
 Island airport
Harbourfront East
 Union Station
 Toronto Islands
Berczy Park
Stn A PO Boxes
Toronto Dominion Centre
 Design Exchange
Commerce Court
 Victoria Hotel
First Canadian Place
 Underground city
Richmond
 Adelaide
 King
St. James Town
Kensington Market
 Chinatown
 Grange Park
Regent Park
 Harbourfront
Garden District
 Ryerson
Central Bay Street
Queen's Park
 Ontario Provincial Government
University of Toronto
 Harbord
Church and Wellesley
St. James Town
 Cabbagetown
Christie
Rosedale
Studio District
Business reply mail Processing Centre
 South Central Letter Processing Plant Toronto
India Bazaar
 The Beaches West
The Beaches
The 

In [8]:
print(toronto_venues.shape)
toronto_venues.head()

(3180, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,The Annex,43.67271,-79.405678,Roti Cuisine of India,43.674618,-79.408249,Indian Restaurant
1,The Annex,43.67271,-79.405678,Ezra's Pound,43.675153,-79.405858,Café
2,The Annex,43.67271,-79.405678,Rose & Sons,43.675668,-79.403617,American Restaurant
3,The Annex,43.67271,-79.405678,Jean Sibelius Square,43.671426,-79.408831,Park
4,The Annex,43.67271,-79.405678,Krispy Kreme Doughnut Cafe,43.674732,-79.407730,Donut Shop


# New York

In [9]:
!wget -q -O 'newyork_data.json' https://cocl.us/new_york_dataset

with open('newyork_data.json') as json_data:
        newyork_data = json.load(json_data)

In [10]:
neighborhoods_data = newyork_data['features']
neighborhoods_data[0]

column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 
neighborhoods = pd.DataFrame(columns=column_names)
for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)


df_ny=neighborhoods
print(df_ny.shape)
df_ny.head()


(306, 4)


,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


In [11]:
address = 'New York City, NY'
geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

# create map of New York using latitude and longitude values
map_newyork = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_ny['Latitude'], df_ny['Longitude'], df_ny['Borough'], df_ny['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_newyork)  
    
map_newyork

In [12]:
ny_venues = getNearbyVenues(names=df_ny['Neighborhood'],
                                   latitudes=df_ny['Latitude'],
                                   longitudes=df_ny['Longitude']
                                  )

Wakefield
Co-op City
Eastchester
Fieldston
Riverdale
Kingsbridge
Marble Hill
Woodlawn
Norwood
Williamsbridge
Baychester
Pelham Parkway
City Island
Bedford Park
University Heights
Morris Heights
Fordham
East Tremont
West Farms
High  Bridge
Melrose
Mott Haven
Port Morris
Longwood
Hunts Point
Morrisania
Soundview
Clason Point
Throgs Neck
Country Club
Parkchester
Westchester Square
Van Nest
Morris Park
Belmont
Spuyten Duyvil
North Riverdale
Pelham Bay
Schuylerville
Edgewater Park
Castle Hill
Olinville
Pelham Gardens
Concourse
Unionport
Edenwald
Bay Ridge
Bensonhurst
Sunset Park
Greenpoint
Gravesend
Brighton Beach
Sheepshead Bay
Manhattan Terrace
Flatbush
Crown Heights
East Flatbush
Kensington
Windsor Terrace
Prospect Heights
Brownsville
Williamsburg
Bushwick
Bedford Stuyvesant
Brooklyn Heights
Cobble Hill
Carroll Gardens
Red Hook
Gowanus
Fort Greene
Park Slope
Cypress Hills
East New York
Starrett City
Canarsie
Flatlands
Mill Island
Manhattan Beach
Coney Island
Bath Beach
Borough Park
Dyker

In [13]:
print(ny_venues.shape)
ny_venues.head()

(9943, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Wakefield,40.894705,-73.847201,Lollipops Gelato,40.894123,-73.845892,Dessert Shop
1,Wakefield,40.894705,-73.847201,Carvel Ice Cream,40.890487,-73.848568,Ice Cream Shop
2,Wakefield,40.894705,-73.847201,Walgreens,40.896528,-73.844700,Pharmacy
3,Wakefield,40.894705,-73.847201,Rite Aid,40.896649,-73.844846,Pharmacy
4,Wakefield,40.894705,-73.847201,Dunkin',40.890459,-73.849089,Donut Shop


# Zurich

In [29]:
wiki_z='https://en.wikipedia.org/wiki/Subdivisions_of_Z%C3%BCrich'
df = pd.read_html(wiki_z)[1]
df=pd.DataFrame(df)
df= df[df.District.str.get(0).isin(['D'])]
df.drop(['Location', 'Coat of Arms', 'BFS-Code', 'Incorporation', 'Areain km²','Population2005','Non-Swiss Citizens'],axis=1,inplace=True)
df.reset_index(drop=True, inplace=True)
df.head(20)

,District,Neighborhood
0,District 1Altstadt,Rathaus Hochschulen Lindenhof City
1,District 2,Wollishofen Leimbach Enge
2,District 3Wiedikon,Alt-Wiedikon Friesenberg Sihlfeld
3,District 4Aussersihl,Werd Langstrasse Hard
4,District 5Industriequartier,Gewerbeschule Escher Wyss
5,District 6,Unterstrass Oberstrass
6,District 7,Fluntern Hottingen Hirslanden Witikon
7,District 8Riesbach,Seefeld Mühlebach Weinegg
8,District 9,Albisrieden Altstetten
9,District 10,Höngg Wipkingen


In [30]:
#rename District name
town=['Zurich','Zurich','Zurich','Zurich','Zurich','Zurich','Zurich','Zurich','Zurich','Zurich','Zurich','Zurich']
df['Town']=town
borough=['Altstadt', 'Enge & Wollishofen', 'Wiedikon', 'Aussersihl', 'Industriequartier', 'Unterstrass', 'Zurich Hill', 'Riesbach', 'Albisrieden', 'Höngg', 'Zürich Nord','Schwamendingen']
df['Borough']=borough
df.drop(['District'],axis=1,inplace=True)
cols = df.columns.tolist()
cols = cols[-1:] + cols[:-1]

df=df[cols]
df.head(12)

,Borough,Neighborhood,Town
0,Altstadt,Rathaus Hochschulen Lindenhof City,Zurich
1,Enge & Wollishofen,Wollishofen Leimbach Enge,Zurich
2,Wiedikon,Alt-Wiedikon Friesenberg Sihlfeld,Zurich
3,Aussersihl,Werd Langstrasse Hard,Zurich
4,Industriequartier,Gewerbeschule Escher Wyss,Zurich
5,Unterstrass,Unterstrass Oberstrass,Zurich
6,Zurich Hill,Fluntern Hottingen Hirslanden Witikon,Zurich
7,Riesbach,Seefeld Mühlebach Weinegg,Zurich
8,Albisrieden,Albisrieden Altstetten,Zurich
9,Höngg,Höngg Wipkingen,Zurich


In [31]:
from geopy.extra.rate_limiter import RateLimiter

locator=Nominatim(user_agent='myGeocoder')
df['Adress']=df['Borough']+ ', '+ df['Town']
# 1 - conveneint function to delay between geocoding calls
geocode = RateLimiter(locator.geocode, min_delay_seconds=1)
# 2- - create location column
df['location'] = df['Adress'].apply(geocode)
# 3 - create longitude, latitude and altitude from location column (returns tuple)
df['point'] = df['location'].apply(lambda loc: tuple(loc.point) if loc else None)
# 4 - split point column into latitude, longitude and altitude columns
df[['Latitude', 'Longitude', 'altitude']] = pd.DataFrame(df['point'].tolist(), index=df.index)
df = df.drop(['Town', 'Adress', 'location','altitude', 'point'], axis=1)
df.head()

,Borough,Neighborhood,Latitude,Longitude
0,Altstadt,Rathaus Hochschulen Lindenhof City,47.372233,8.542329
1,Enge & Wollishofen,Wollishofen Leimbach Enge,47.351598,8.526223
2,Wiedikon,Alt-Wiedikon Friesenberg Sihlfeld,47.366267,8.510713
3,Aussersihl,Werd Langstrasse Hard,47.378714,8.521200
4,Industriequartier,Gewerbeschule Escher Wyss,47.387507,8.520627


In [38]:
df=pd.DataFrame(df)
df_zurich=df.set_index(['Borough','Latitude','Longitude']).stack().str.split(' ', expand=True).stack()
df_zurich=df_zurich.unstack(-2).reset_index(-1,drop=True).reset_index()
df_zurich= df_zurich.reindex(columns=['Borough','Neighborhood','Latitude', 'Longitude'])
print(df_zurich.shape)
df_zurich.head()

(36, 4)


,Borough,Neighborhood,Latitude,Longitude
0,Albisrieden,Albisrieden,47.374857,8.484657
1,Albisrieden,Altstetten,47.374857,8.484657
2,Altstadt,Rathaus,47.372233,8.542329
3,Altstadt,Hochschulen,47.372233,8.542329
4,Altstadt,Lindenhof,47.372233,8.542329


In [18]:
address = 'Zurich,CH'

geolocator = Nominatim(user_agent="to_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
# create map of New York using latitude and longitude values
map_zurich = folium.Map(location=[latitude, longitude], zoom_start=12)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_zurich['Latitude'], df_zurich['Longitude'], df_zurich['Borough'], df_zurich['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_zurich)  
    
map_zurich

In [19]:
zurich_venues = getNearbyVenues(names=df_zurich['Neighborhood'],
                                   latitudes=df_zurich['Latitude'],
                                   longitudes=df_zurich['Longitude']
                                  )

Rathaus Hochschulen Lindenhof City
Wollishofen Leimbach Enge
Alt-Wiedikon Friesenberg Sihlfeld
Werd Langstrasse Hard
Gewerbeschule Escher Wyss
Unterstrass Oberstrass
Fluntern Hottingen Hirslanden Witikon
Seefeld Mühlebach Weinegg
Albisrieden Altstetten
Höngg Wipkingen
Affoltern Oerlikon Seebach
Saatlen Schwamendingen Mitte Hirzenbach


In [20]:
print(zurich_venues.shape)
zurich_venues.head()

(443, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Rathaus Hochschulen Lindenhof City,47.372233,8.542329,Lindenhof,47.373005,8.540883,Pedestrian Plaza
1,Rathaus Hochschulen Lindenhof City,47.372233,8.542329,Old Crow,47.372092,8.541024,Cocktail Bar
2,Rathaus Hochschulen Lindenhof City,47.372233,8.542329,Widder Bar,47.372415,8.539863,Hotel Bar
3,Rathaus Hochschulen Lindenhof City,47.372233,8.542329,Schwarzenbach Kolonialwaren,47.371444,8.544091,Gourmet Shop
4,Rathaus Hochschulen Lindenhof City,47.372233,8.542329,Café Schober,47.371400,8.544149,Café


Top 3 venues
comparaison n. de venues n. de neighborhood
nombre de différente venues -> est-ce que zurich a beaucoup moins à proposer


[47.410307912890964,
 8.533326309279047,
 47.347706193148156,
 8.52778442275121,
 47.405354341095375,
 8.578704410848227,
 47.375043253458934,
 8.478472353861012]